# Import Dataset Hourly & Daily V2

This notebook fetches comprehensive weather data from Open-Meteo Archive API with extended features.

## Features Added in V2:

### Hourly Features (11 total):
- `temperature_2m`, `weather_code`, `relative_humidity_2m`, `pressure_msl`, `wind_speed_10m`
- `rain`, `apparent_temperature`, `surface_pressure`, `dew_point_2m`, `wind_direction_10m`, `wind_gusts_10m`

### Daily Features (21 total):
- Temperature: `temperature_2m_max`, `temperature_2m_min`, `temperature_2m_mean`
- Humidity: `relative_humidity_2m_mean`, `relative_humidity_2m_max`, `relative_humidity_2m_min`
- Pressure (MSL): `pressure_msl_mean`, `pressure_msl_max`, `pressure_msl_min`
- Surface Pressure: `surface_pressure_mean`, `surface_pressure_max`, `surface_pressure_min`
- Wind Speed: `wind_speed_10m_mean`, `wind_speed_10m_max`, `wind_speed_10m_min`
- Wind Gusts: `wind_gusts_10m_mean`, `wind_gusts_10m_max`, `wind_gusts_10m_min`
- Other: `weather_code`, `rain_sum`, `winddirection_10m_dominant`

In [1]:
import requests
import pandas as pd
from datetime import datetime
import time
import os

# Coordinates and Parameters
LATITUDE = -7.0520702239386175
LONGITUDE = 110.43532807750137
TIMEZONE = "Asia/Jakarta"
API_URL = "https://archive-api.open-meteo.com/v1/archive"

# Rate Limiting Config
MAX_RETRIES = 5
BASE_DELAY = 2  # Base delay between requests (seconds)
RETRY_DELAY = 30  # Initial retry delay for 429 errors (seconds)

# Extended Feature Lists for V2
HOURLY_FEATURES = [
    "temperature_2m", "weather_code", "relative_humidity_2m", "pressure_msl",
    "wind_speed_10m", "rain", "apparent_temperature", "surface_pressure",
    "dew_point_2m", "wind_direction_10m", "wind_gusts_10m"
]

DAILY_FEATURES = [
    "temperature_2m_max", "temperature_2m_min", "weather_code",
    "relative_humidity_2m_mean", "pressure_msl_mean", "wind_speed_10m_mean",
    "temperature_2m_mean", "relative_humidity_2m_max", "relative_humidity_2m_min",
    "pressure_msl_max", "pressure_msl_min", "wind_speed_10m_max",
    "rain_sum", "surface_pressure_mean", "surface_pressure_max", "surface_pressure_min",
    "winddirection_10m_dominant", "wind_gusts_10m_min", "wind_speed_10m_min",
    "wind_gusts_10m_max", "wind_gusts_10m_mean"
]

print(f"Hourly features: {len(HOURLY_FEATURES)}")
print(f"Daily features: {len(DAILY_FEATURES)}")

Hourly features: 11
Daily features: 21


In [2]:
# Weather Condition Mapping (Optional helper, main logic uses weather_code)
def map_weather_code(code):
    """Maps WMO weather code to user's custom condition string."""
    if code is None:
        return 'Unknown'
    if code == 0:
        return 'Clear'
    elif code in [1, 2]:
        return 'Partially cloudy'
    elif code in [3, 45, 48]:
        return 'Overcast'
    elif code in [51, 53, 55]:
        return 'Rain'
    elif code in [61, 63, 65]:
        return 'Rain, Overcast'
    elif code in [80, 81, 82]:
        return 'Rain, Partially cloudy'
    elif code in [95, 96, 99]:
        return 'Rain'
    else:
        return 'Unknown'

In [3]:
def fetch_hourly_data_chunk(start_date, end_date, retries=0):
    """Fetch hourly and daily data for a specific date range with retry logic."""
    params = {
        "latitude": LATITUDE,
        "longitude": LONGITUDE,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": HOURLY_FEATURES,
        "daily": DAILY_FEATURES,
        "timezone": TIMEZONE
    }
    
    try:
        response = requests.get(API_URL, params=params)
        
        # Handle rate limiting (429)
        if response.status_code == 429:
            if retries < MAX_RETRIES:
                wait_time = RETRY_DELAY * (2 ** retries)  # Exponential backoff
                print(f"   ⚠️ Rate limited! Waiting {wait_time}s before retry {retries + 1}/{MAX_RETRIES}...")
                time.sleep(wait_time)
                return fetch_hourly_data_chunk(start_date, end_date, retries + 1)
            else:
                print(f"   ❌ Max retries reached for {start_date} to {end_date}")
                return None
        
        response.raise_for_status()
        return response.json()
        
    except requests.exceptions.RequestException as e:
        if retries < MAX_RETRIES:
            wait_time = RETRY_DELAY * (2 ** retries)
            print(f"   ⚠️ Error: {e}. Retrying in {wait_time}s...")
            time.sleep(wait_time)
            return fetch_hourly_data_chunk(start_date, end_date, retries + 1)
        else:
            print(f"   ❌ Failed after {MAX_RETRIES} retries: {e}")
            return None

In [4]:
def fetch_historical_hourly_data():
    """Fetch hourly and daily data from 2000 to today in yearly chunks."""
    today = datetime.now()
    start_year = 2000
    end_year = today.year
    
    all_data = []
    total_years = end_year - start_year + 1
    
    for idx, year in enumerate(range(start_year, end_year + 1)):
        start_date = f"{year}-01-01"
        if year == end_year:
            end_date = today.strftime("%Y-%m-%d")
        else:
            end_date = f"{year}-12-31"
        
        print(f"[{idx + 1}/{total_years}] Fetching data for {year} ({start_date} to {end_date})...")
        
        data = fetch_hourly_data_chunk(start_date, end_date)
        
        if data is None:
            print(f"   ⏭️ Skipping year {year} due to errors")
            continue
            
        hourly_data = data.get("hourly", {})
        daily_data = data.get("daily", {})
        
        if not hourly_data:
            print(f"   ⚠️ No hourly data found for {year}.")
            continue
        
        # Process hourly data with extended features
        df_hourly = pd.DataFrame({
            "timestamp": hourly_data["time"],
            "temp": hourly_data.get("temperature_2m"),
            "humidity": hourly_data.get("relative_humidity_2m"),
            "windspeed": hourly_data.get("wind_speed_10m"),
            "sealevelpressure": hourly_data.get("pressure_msl"),
            "weather_code": hourly_data.get("weather_code"),
            "rain": hourly_data.get("rain"),
            "apparent_temperature": hourly_data.get("apparent_temperature"),
            "surface_pressure": hourly_data.get("surface_pressure"),
            "dew_point_2m": hourly_data.get("dew_point_2m"),
            "wind_direction_10m": hourly_data.get("wind_direction_10m"),
            "wind_gusts_10m": hourly_data.get("wind_gusts_10m")
        })
        df_hourly["timestamp"] = pd.to_datetime(df_hourly["timestamp"])
        df_hourly["date_only"] = df_hourly["timestamp"].dt.date  # Helper for merging

        # Process daily data with extended features
        df_daily = pd.DataFrame({
            "date_only": daily_data.get("time"),
            # Temperature
            "temp_max_daily": daily_data.get("temperature_2m_max"),
            "temp_min_daily": daily_data.get("temperature_2m_min"),
            "temp_mean_daily": daily_data.get("temperature_2m_mean"),
            # Weather
            "weather_code_daily": daily_data.get("weather_code"),
            "rain_sum_daily": daily_data.get("rain_sum"),
            # Humidity
            "humidity_mean_daily": daily_data.get("relative_humidity_2m_mean"),
            "humidity_max_daily": daily_data.get("relative_humidity_2m_max"),
            "humidity_min_daily": daily_data.get("relative_humidity_2m_min"),
            # MSL Pressure
            "pressure_msl_mean_daily": daily_data.get("pressure_msl_mean"),
            "pressure_msl_max_daily": daily_data.get("pressure_msl_max"),
            "pressure_msl_min_daily": daily_data.get("pressure_msl_min"),
            # Surface Pressure
            "surface_pressure_mean_daily": daily_data.get("surface_pressure_mean"),
            "surface_pressure_max_daily": daily_data.get("surface_pressure_max"),
            "surface_pressure_min_daily": daily_data.get("surface_pressure_min"),
            # Wind Speed
            "windspeed_mean_daily": daily_data.get("wind_speed_10m_mean"),
            "windspeed_max_daily": daily_data.get("wind_speed_10m_max"),
            "windspeed_min_daily": daily_data.get("wind_speed_10m_min"),
            # Wind Gusts
            "wind_gusts_mean_daily": daily_data.get("wind_gusts_10m_mean"),
            "wind_gusts_max_daily": daily_data.get("wind_gusts_10m_max"),
            "wind_gusts_min_daily": daily_data.get("wind_gusts_10m_min"),
            # Wind Direction
            "wind_direction_dominant_daily": daily_data.get("winddirection_10m_dominant")
        })
        df_daily["date_only"] = pd.to_datetime(df_daily["date_only"]).dt.date

        # Merge hourly and daily data
        df_year = pd.merge(df_hourly, df_daily, on="date_only", how="left")
        df_year = df_year.drop(columns=["date_only"])  # Drop helper column
        
        all_data.append(df_year)
        print(f"   ✅ {len(df_year):,} records fetched")
        
        # Longer delay between requests to avoid rate limiting
        if idx < total_years - 1:  # Don't wait after last request
            print(f"   ⏳ Waiting {BASE_DELAY}s before next request...")
            time.sleep(BASE_DELAY)
    
    if not all_data:
        print("❌ No data fetched.")
        return None
    
    df = pd.concat(all_data, ignore_index=True)
    
    # Add time-based features
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    df["year"] = df["timestamp"].dt.year
    
    df["conditions"] = df["weather_code"].apply(map_weather_code)
    df["id"] = range(len(df))
    
    # Define output columns (organized by category)
    output_columns = [
        # Identifiers
        "id", "timestamp", "hour", "day", "month", "year",
        # Hourly Features
        "temp", "humidity", "windspeed", "sealevelpressure", "rain",
        "apparent_temperature", "surface_pressure", "dew_point_2m",
        "wind_direction_10m", "wind_gusts_10m",
        "weather_code", "conditions",
        # Daily Temperature
        "temp_max_daily", "temp_min_daily", "temp_mean_daily",
        # Daily Weather
        "weather_code_daily", "rain_sum_daily",
        # Daily Humidity
        "humidity_mean_daily", "humidity_max_daily", "humidity_min_daily",
        # Daily MSL Pressure
        "pressure_msl_mean_daily", "pressure_msl_max_daily", "pressure_msl_min_daily",
        # Daily Surface Pressure
        "surface_pressure_mean_daily", "surface_pressure_max_daily", "surface_pressure_min_daily",
        # Daily Wind Speed
        "windspeed_mean_daily", "windspeed_max_daily", "windspeed_min_daily",
        # Daily Wind Gusts
        "wind_gusts_mean_daily", "wind_gusts_max_daily", "wind_gusts_min_daily",
        # Daily Wind Direction
        "wind_direction_dominant_daily"
    ]
    
    final_df = df[output_columns]
    
    return final_df

In [5]:
# Execute the data fetch
df = fetch_historical_hourly_data()

if df is not None:
    # Ensure output directory exists
    output_dir = "../data"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_file = os.path.join(output_dir, "historical_data_2000_2024_v2.csv")
    df.to_csv(output_file, index=False)
    print(f"\n🎉 Data successfully saved to {output_file}")
    print(f"📊 Total records: {len(df):,}")
    print(f"📋 Total columns: {len(df.columns)}")

[1/26] Fetching data for 2000 (2000-01-01 to 2000-12-31)...
   ✅ 8,784 records fetched
   ⏳ Waiting 2s before next request...
[2/26] Fetching data for 2001 (2001-01-01 to 2001-12-31)...
   ✅ 8,760 records fetched
   ⏳ Waiting 2s before next request...
[3/26] Fetching data for 2002 (2002-01-01 to 2002-12-31)...
   ✅ 8,760 records fetched
   ⏳ Waiting 2s before next request...
[4/26] Fetching data for 2003 (2003-01-01 to 2003-12-31)...
   ✅ 8,760 records fetched
   ⏳ Waiting 2s before next request...
[5/26] Fetching data for 2004 (2004-01-01 to 2004-12-31)...
   ✅ 8,784 records fetched
   ⏳ Waiting 2s before next request...
[6/26] Fetching data for 2005 (2005-01-01 to 2005-12-31)...
   ✅ 8,760 records fetched
   ⏳ Waiting 2s before next request...
[7/26] Fetching data for 2006 (2006-01-01 to 2006-12-31)...
   ⚠️ Error: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=None). Retrying in 30s...
   ✅ 8,760 records fetched
   ⏳ Waiting 2s before

In [6]:
# Preview the data
if df is not None:
    print("\n=== Column List ===")
    for i, col in enumerate(df.columns):
        print(f"{i+1:2}. {col}")
    
    print("\n=== Data Preview (First 5 rows) ===")
    display(df.head())
    
    print("\n=== Data Info ===")
    df.info()


=== Column List ===
 1. id
 2. timestamp
 3. hour
 4. day
 5. month
 6. year
 7. temp
 8. humidity
 9. windspeed
10. sealevelpressure
11. rain
12. apparent_temperature
13. surface_pressure
14. dew_point_2m
15. wind_direction_10m
16. wind_gusts_10m
17. weather_code
18. conditions
19. temp_max_daily
20. temp_min_daily
21. temp_mean_daily
22. weather_code_daily
23. rain_sum_daily
24. humidity_mean_daily
25. humidity_max_daily
26. humidity_min_daily
27. pressure_msl_mean_daily
28. pressure_msl_max_daily
29. pressure_msl_min_daily
30. surface_pressure_mean_daily
31. surface_pressure_max_daily
32. surface_pressure_min_daily
33. windspeed_mean_daily
34. windspeed_max_daily
35. windspeed_min_daily
36. wind_gusts_mean_daily
37. wind_gusts_max_daily
38. wind_gusts_min_daily
39. wind_direction_dominant_daily

=== Data Preview (First 5 rows) ===


,id,timestamp,hour,day,month,year,temp,humidity,windspeed,sealevelpressure,...,surface_pressure_mean_daily,surface_pressure_max_daily,surface_pressure_min_daily,windspeed_mean_daily,windspeed_max_daily,windspeed_min_daily,wind_gusts_mean_daily,wind_gusts_max_daily,wind_gusts_min_daily,wind_direction_dominant_daily
0,0,2000-01-01 00:00:00,0,1,1,2000,21.8,98,4.0,1008.4,...,983.6,985.4,981.3,6.3,12.1,1.4,17.3,30.2,6.8,300
1,1,2000-01-01 01:00:00,1,1,1,2000,21.4,99,4.0,1007.9,...,983.6,985.4,981.3,6.3,12.1,1.4,17.3,30.2,6.8,300
2,2,2000-01-01 02:00:00,2,1,1,2000,21.4,98,3.2,1007.4,...,983.6,985.4,981.3,6.3,12.1,1.4,17.3,30.2,6.8,300
3,3,2000-01-01 03:00:00,3,1,1,2000,21.2,99,4.6,1007.0,...,983.6,985.4,981.3,6.3,12.1,1.4,17.3,30.2,6.8,300
4,4,2000-01-01 04:00:00,4,1,1,2000,21.0,99,3.6,1006.9,...,983.6,985.4,981.3,6.3,12.1,1.4,17.3,30.2,6.8,300



=== Data Info ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227328 entries, 0 to 227327
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   id                             227328 non-null  int64         
 1   timestamp                      227328 non-null  datetime64[ns]
 2   hour                           227328 non-null  int32         
 3   day                            227328 non-null  int32         
 4   month                          227328 non-null  int32         
 5   year                           227328 non-null  int32         
 6   temp                           227328 non-null  float64       
 7   humidity                       227328 non-null  int64         
 8   windspeed                      227328 non-null  float64       
 9   sealevelpressure               227328 non-null  float64       
 10  rain                           227328 non-null  f

In [7]:
# Summary Statistics
if df is not None:
    print("=== Summary Statistics ===")
    display(df.describe())
    
    print("\n=== Missing Values ===")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    display(missing_df[missing_df['Missing Count'] > 0])

=== Summary Statistics ===


,id,timestamp,hour,day,month,year,temp,humidity,windspeed,sealevelpressure,...,surface_pressure_mean_daily,surface_pressure_max_daily,surface_pressure_min_daily,windspeed_mean_daily,windspeed_max_daily,windspeed_min_daily,wind_gusts_mean_daily,wind_gusts_max_daily,wind_gusts_min_daily,wind_direction_dominant_daily
count,227328.000000,227328,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,...,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000,227328.000000
mean,113663.500000,2012-12-18 23:30:00,11.500000,15.721706,6.508235,2012.466639,25.537447,79.352064,6.158687,1010.092583,...,986.395946,988.209449,984.307770,6.160557,11.302280,1.956123,16.850401,29.357158,6.813936,193.132496
min,0.000000,2000-01-01 00:00:00,0.000000,1.000000,1.000000,2000.000000,17.000000,16.000000,0.000000,1002.200000,...,980.900000,982.100000,978.700000,1.700000,3.700000,0.000000,8.500000,11.500000,0.700000,0.000000
25%,56831.750000,2006-06-25 23:45:00,5.750000,8.000000,4.000000,2006.000000,23.200000,71.000000,3.400000,1008.800000,...,985.500000,987.200000,983.500000,4.300000,8.400000,0.800000,13.500000,24.100000,4.700000,130.000000
50%,113663.500000,2012-12-18 23:30:00,11.500000,16.000000,7.000000,2012.000000,25.000000,83.000000,5.200000,1010.200000,...,986.500000,988.300000,984.400000,5.500000,10.400000,1.500000,15.700000,27.700000,5.800000,158.000000
75%,170495.250000,2019-06-13 23:15:00,17.250000,23.000000,9.000000,2019.000000,27.500000,93.000000,8.000000,1011.400000,...,987.300000,989.200000,985.200000,7.200000,13.300000,2.400000,18.800000,32.800000,7.600000,292.000000
max,227327.000000,2025-12-06 23:00:00,23.000000,31.000000,12.000000,2025.000000,37.800000,100.000000,34.700000,1016.800000,...,990.900000,993.000000,989.400000,26.900000,34.700000,22.900000,58.500000,73.800000,46.400000,360.000000
std,65624.085335,NaN,6.922202,8.802615,3.441810,7.482721,3.021679,16.928284,3.900765,1.867074,...,1.368332,1.470494,1.333127,2.656947,4.188206,1.935618,4.961420,7.465278,3.879874,85.280474



=== Missing Values ===


,Missing Count,Missing %
